# FX vol trading strategy - implementation

In [56]:
# Import libraries
import pandas as pd
import numpy as np
from data_cleaning.import_data import import_data


### Algorithm description (WIP..)

1) For a given trading pair, say USDRUB, fit the appropriate GARCH model for time t = 0 until time t = T
2) Standing at time T, forecast the 30-day volatility. If the forecast is above market pricing, enter long straddle, if below then enter a short straddle. If not signficantly different, do nothing. 

In [57]:
df = import_data(fx_pair='USDRUB')
df.dropna(inplace=True)
df['log_ret'] = df['log_ret'] * 100
df['direction'] = 0

In [58]:
threshold_up = 0.02
threshold_down = -0.02

conditions = [ 
    (df['rolling_21d_realized_stdev'] > threshold_up) , 
    (df['rolling_21d_realized_stdev'] < threshold_down) , 
    (threshold_down <= df['rolling_21d_realized_stdev']) & (df['rolling_21d_realized_stdev'] <= threshold_up) 
    ]
directions = [1, -1, 0]

df['direction'] = np.select(conditions, directions, np.nan)


In [59]:
trade_conditions = [(df['rolling_21d_realized_stdev'] > threshold_up), (df['rolling_21d_realized_stdev'] < threshold_down)]
counter = 0 
df['direction_new'] = 0

for index, row in df.iterrows():
    
    if counter > 0 : 
        counter -= counter
        continue

    if row['rolling_21d_realized_stdev'] > threshold_up:
        row['direction_new'] = 1
        counter = 21

    if row['rolling_21d_realized_stdev'] < threshold_down:
        row['direction_new'] = -1
        counter = 21
    

In [60]:
df.tail(30)

,v1m,v3m,v1y,px_last,px_bid,px_ask,log_ret,rolling_21d_realized_stdev,direction,direction_new
Dates,,,,,,,,,,
2022-01-12,15.2050,15.4650,15.5600,74.7175,74.7050,74.7300,0.344689,0.009421,0.0,0
2022-01-13,16.4900,16.3950,16.1775,76.3048,76.2700,76.3396,2.102151,0.026261,1.0,0
2022-01-14,16.8725,16.6900,16.3150,76.2402,76.2120,76.2684,-0.084696,0.024913,1.0,0
2022-01-17,17.7425,17.1550,16.6200,76.1035,76.0770,76.1300,-0.179463,0.019636,0.0,0
2022-01-18,18.0150,17.5500,17.1000,76.9664,76.9420,76.9909,1.127471,0.027891,1.0,0
2022-01-19,17.9125,17.5900,17.1500,76.2094,76.1947,76.2240,-0.988415,0.022141,1.0,0
2022-01-20,18.1275,17.5700,17.0850,76.6905,76.6810,76.7000,0.629303,0.029330,1.0,0
2022-01-21,18.7300,18.0550,17.3400,77.4677,77.4467,77.4886,1.008323,0.041654,1.0,0
2022-01-24,20.9650,19.7550,18.4000,78.7687,78.7404,78.7970,1.665463,0.050648,1.0,0
